In [1]:
import os
import sys
sys.path.append("/home/pervinco/BKAI_MetaPolyp")

import cv2
import yaml
import random
import numpy as np
import albumentations as A

from glob import glob
from utils.utils import visualize
from data.batch_preprocess import background_pasting, get_file_list, load_img_mask

In [2]:
with open("/home/pervinco/BKAI_MetaPolyp/config.yaml", "r") as f:
    config = yaml.safe_load(f)

data_dir = "/home/pervinco/Datasets/BKAI_IGH_NeoPolyp"
color_dir = f"{data_dir}/color_txt"
image_dir = f"{data_dir}/train"
mask_dir = f"{data_dir}/train_gt"

background_dir = "/home/pervinco/Datasets/Background"
background_folders = ["0_normal", "1_ulcerative_colitis", "3_esophagitis"]
output_dir = "/home/pervinco/Datasets/BKAI_FAKE"

In [3]:
train_color_files = f"{data_dir}/train_colors"
valid_color_files = f"{data_dir}/valid_colors"

train_txt_files = sorted(glob(f"{train_color_files}/*.txt"))
valid_txt_files = sorted(glob(f"{valid_color_files}/*.txt"))
print(len(train_txt_files), len(valid_txt_files))

3 3


In [4]:
if not os.path.isdir(output_dir):
    os.makedirs(f"{output_dir}/train", exist_ok=True)
    os.makedirs(f"{output_dir}/train_gt", exist_ok=True)
    os.makedirs(f"{output_dir}/train_colors", exist_ok=True)
    os.makedirs(f"{output_dir}/valid_colors", exist_ok=True)

In [5]:
total_valid = {}
for val_txt in valid_txt_files:
    name = val_txt.split('/')[-1].split('.')[0]

    with open(val_txt, "r") as f:
        files = [x.strip() for x in f.readlines()]
    
    total_valid.update({name : files})

print(len(total_valid))

valid_file_list = []
for name, files in total_valid.items():
    valid_file_list.extend(files)

print(len(valid_file_list))

3
200


In [6]:
total_files = get_file_list(color_dir)

for name, files in total_files.items():
    print(name, len(files))

green 257
red 694
rng 49


In [7]:
backgrounds = []
for folder in background_folders:
    files = sorted(glob(f"{background_dir}/*/{folder}/*"))
    backgrounds.extend(files)

print(len(backgrounds))

4500


In [8]:
color_per_iter = {"red" : config["red_size"], "green" : config["green_size"], "rng" : config["rng_size"]}

In [9]:
for name, files in total_files.items():
    with open(f"{output_dir}/train_colors/{name}.txt", "w") as f:
        for file in files:
            image_path = f"{image_dir}/{file}.jpeg"
            mask_path = f"{mask_dir}/{file}.jpeg"
            image, mask = load_img_mask(image_path, mask_path, size=config["img_size"])
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            # mask = cv2.cvtColor(mask, cv2.COLOR_BGR2RGB)

            if file not in valid_file_list:
                f.write(f"{file}\n")

            cv2.imwrite(f"{output_dir}/train/{file}.jpeg", image)
            cv2.imwrite(f"{output_dir}/train_gt/{file}.jpeg", mask)

            for idx in range(0, color_per_iter[name]):
                bg_idx = random.randint(0, len(backgrounds) - 1)
                background_file = backgrounds[bg_idx]
                background_image = cv2.imread(background_file)
                background_image = cv2.resize(background_image, (config["img_size"], config["img_size"]))
                # background_image = cv2.cvtColor(background_image, cv2.COLOR_BGR2RGB)

                result_image, result_mask = background_pasting(image, mask, background_image, alpha=0.6)
                # result_image = cv2.cvtColor(result_image, cv2.COLOR_BGR2RGB)
                # result_mask = cv2.cvtColor(result_mask, cv2.COLOR_BGR2RGB)

                f.write(f"{file}_{idx:>02}\n")

                cv2.imwrite(f"{output_dir}/train/{file}_{idx:>02}.jpeg", result_image)
                cv2.imwrite(f"{output_dir}/train_gt/{file}_{idx:>02}.jpeg", result_mask)
    

In [10]:
for val_txt in valid_txt_files:
    name = val_txt.split('/')[-1].split('.')[0]

    with open(val_txt, "r") as f:
        files = [x.strip() for x in f.readlines()]
    
    with open(f"{output_dir}/valid_colors/{name}.txt", "w") as f:
        for file in files:
            f.write(f"{file}\n")